In [1]:
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import numpy as np
import math
from rouge import Rouge 
import string

In [2]:
#Metinleri direkt olarak txt üzerinden okuyorum ancak sizin için daha anlaşılır olması adına direkt olarak texti kopyaladım.
text="""Dollar gains on Greenspan speech

The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.

And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next."

Worries about the deficit concerns about China do, however, remain. China's currency remains pegged to the dollar and the US currency's sharp falls in recent months have therefore made Chinese export prices highly competitive. But calls for a shift in Beijing's policy have fallen on deaf ears, despite recent comments in a major Chinese newspaper that the "time is ripe" for a loosening of the peg. The G7 meeting is thought unlikely to produce any meaningful movement in Chinese policy. In the meantime, the US Federal Reserve's decision on 2 February to boost interest rates by a quarter of a point - the sixth such move in as many months - has opened up a differential with European rates. The half-point window, some believe, could be enough to keep US assets looking more attractive, and could help prop up the dollar. The recent falls have partly been the result of big budget deficits, as well as the US's yawning current account gap, both of which need to be funded by the buying of US bonds and assets by foreign firms and governments. The White House will announce its budget on Monday, and many commentators believe the deficit will remain at close to half a trillion dollars.
"""

In [3]:
tokenize_sent = sent_tokenize(text) 
#Noktalama işaretlerinin kaldırılması işlemleri 
import re
for i in range(0,len(tokenize_sent)):
        tokenize_sent[i]=re.sub(r'[^\w\s]','',tokenize_sent[i])

In [4]:
adj_matrix= np.zeros((len(tokenize_sent),len(tokenize_sent))) #Cümle sayısına göre 
sent_rank = np.random.rand(len(tokenize_sent)) #Cümle değerleri için tuttuğum değişken

#Stopword'lerin kaldırılması
stop = stopwords.words('english')
process_sent=list()
    
for i in range(0,len(tokenize_sent)):
    if i not in stop:
        process_sent.append(tokenize_sent[i].split())

In [5]:
freq=dict()
copy_sent=process_sent

In [6]:
#Stemming işlemi 
from nltk.stem import PorterStemmer 
ps= PorterStemmer()
for sentence_in_running,i in enumerate(process_sent):
    for word_in_running,j in enumerate(i):
        copy_sent[sentence_in_running][word_in_running]=ps.stem(j)

In [7]:
#Kelime sıklığı
for sentence in copy_sent:
    for word in sentence:
        if word in freq:
            freq[word]=freq[word]+1
        else :
            freq[word]=1
            

In [8]:
#Matris için kenarların belirlenmesi işlemleri, Distortion

def calculate_different(i,j):
    find_diff=0
    point=0
    sent_i=copy_sent[i]
    sent_j=copy_sent[j]
    for word in sent_i:
        test_diff=sent_j.count(word)  #2. cümlede şuan için işlenen kelimenin geçme sıklığı
        point=(freq[word]-test_diff)+point #Toplam sıklıktan 2. cümle sıklığını düşürerek cümleler arası puanlama
        if(test_diff==0): # Eğer kelime benzer değilse farklılık +1 
            find_diff=find_diff+1 
    for word in sent_j: # 2. cümlede bulunup, 1. cümlede olmayan kelimeleri, puana ekliyorum aynı zamanda farklılık değişkenini artırıyyorum
        if word not in sent_i:
            point=point+freq[word]
            find_diff=find_diff+1
    if(find_diff==0):
        find_diff=1
    different=(point/float(find_diff))*2 # Terimlerin sıklıkları üzerinden hesaplanan puan ve farklı kelime sayısına göre bir kenar değeri elde ediyorum
    return different

In [9]:
#Yukarıda bulunan fonksiyonu metinde geçen her kelime için tekrarlama
for i in range(0,len(tokenize_sent)):
    for j in range(i+1,len(tokenize_sent)):
        adj_matrix[i,j] = adj_matrix[j,i] = calculate_different(i,j)

In [10]:
cümleler={}

In [11]:
#Her cümle için kendi içinde bulunan kelimelerin sıklığını almak istiyorum. Buna göre Node değerlerini belirlemek için
for i in range(len(copy_sent)):
    cümleler[i]=0

In [14]:
for i in range(0, len(copy_sent)):
    for j in range(0,len(copy_sent[i])):
        cümleler[i]=cümleler[i]+freq[copy_sent[i][j]]

In [15]:
cümleler

{0: 458,
 1: 204,
 2: 212,
 3: 202,
 4: 398,
 5: 334,
 6: 210,
 7: 110,
 8: 282,
 9: 344,
 10: 138,
 11: 430,
 12: 240,
 13: 510,
 14: 280}

In [18]:
#Ranking işlemi. Node her cümleyi temsil ediyor. Her cümlenin puanını hesaplarken aynı zamanda diğer cümlelerde bulunan 
#farklılık değerlerini de göz önünde bulunduruyorum. Karesini alma işlemini eşitlik durumlarını en aza indirmek için yaptım
def calculate_weight(sent_rank,adjacency,shp,cümleler):
    sent_rank = np.array(sent_rank)
    for node1 in range(0,shp[0]):
        sent_rank[node1] = 0
        for node2 in range(0,shp[0]):
            sent_rank[node1] = (adj_matrix[node1,node2]*cümleler[node1])**2+sent_rank[node1]
            
    return sent_rank

In [19]:
sent_rank = calculate_weight(sent_rank,adj_matrix,adj_matrix.shape,cümleler)
sentences_rank=(sent_rank).argsort()

In [20]:
#Yukarıda her cümle için tek bir değer oluşturdum ve bunları sıraladım. Aşağıda artık sıralanan cümleler arasında en yüksek değerli
#Cümleleri seçme ve yazdırma işlemlerini gerçekleştirdim
summarization_sorted=np.sort(sentences_rank[1:7])
for i in range(0,len(summarization_sorted)):
    print(tokenize_sent[summarization_sorted[i]])

And Alan Greenspan highlighted the US governments willingness to curb spending and rising household savings as factors which may help to reduce it
In late trading in New York the dollar reached 12871 against the euro from 12974 on Thursday
Market concerns about the deficit has hit the greenback in recent months
Hes taking a longerterm view laying out a set of conditions under which the current account deficit can improve this year and next
The G7 meeting is thought unlikely to produce any meaningful movement in Chinese policy
The halfpoint window some believe could be enough to keep US assets looking more attractive and could help prop up the dollar


In [45]:
#Rouge-Score
hypothesis1="""And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it.In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday.Market concerns about the deficit has hit the greenback in recent months.On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data."I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time, said Robert Sinche, head of currency strategy at Bank of America in New York."He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next."""
reference1=""""The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.China's currency remains pegged to the dollar and the US currency's sharp falls in recent months have therefore made Chinese export prices highly competitive.Market concerns about the deficit has hit the greenback in recent months."I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York.The recent falls have partly been the result of big budget deficits, as well as the US's yawning current account gap, both of which need to be funded by the buying of US bonds and assets by foreign firms and governments."He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next."""
rouge = Rouge()
scores = rouge.get_scores(hypothesis1, reference1)
scores

[{'rouge-1': {'f': 0.5999999950133196,
   'p': 0.6326530612244898,
   'r': 0.5705521472392638},
  'rouge-2': {'f': 0.4480519430654411,
   'p': 0.4726027397260274,
   'r': 0.42592592592592593},
  'rouge-l': {'f': 0.5585585535589644,
   'p': 0.5535714285714286,
   'r': 0.5636363636363636}}]